# Init

In [ ]:
# Variable

eta = float;

xlist = list;
dList = list;

delWMatrix = list;
wMatrix = list;
vMatrix = list;
yMatrix = list;
sigmaMatrix = list;
nodeIndexMatrix = list;

## Functions

### Calculation Function

In [ ]:
def eFunc(layerNum:int,nodeNum:int):
    pass;

def EFunc(nodeNum:int):
    pass;

def EavFunc():
    pass;

def vFunc(layerNum:int,nodeNum:int):
    pass;

def yFunc(layerNum:int,nodeNum:int):
    pass;

def activateFunc(input:int):
    pass;

def activatePrimeFunc(input:int):
    pass;

def sigmaFunc(layerNum:int,nodeNum:int):
    pass;



### Auxilary Function

In [ ]:
def initContainer(hlCount:int):
    pass;

def getRandomWList():
    pass;

def loadXList(index:int):
    pass;

def loadDList(index:int):
    pass;

# Training Ground

## Training Function

In [ ]:
def getAllE():
    pass;

def backPropergate():
    pass;

## Execution

In [ ]:
# input attribute value

# calculate hidden layer one

# calculate final layer

# keep set of results 

# Evaluation

In [1]:
# X = [1,2,3]
# Y = [10,20,30]
# Z = X + Y 
# print(Z)

[1, 2, 3, 10, 20, 30]
